In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as LA

In [2]:
# 1. 데이터 불러오기 및 전처리
# [중요] 'Copy Path'로 복사한 실제 경로를 r' ' 안에 넣으세요.
file_path = r'c:\\Users\\amro0\\.vscode\\lab\\item__로그수익률.parquet'
df = pd.read_parquet(file_path)

In [10]:
# 2. nan 해결을 위한 강력한 정제
# (1) 모든 값이 결측치인 종목(컬럼)은 아예 삭제합니다.
df_clean = df.dropna(axis=1, how='all')

# (2) 그래도 남은 결측치는 0(수익률 변동 없음)으로 채웁니다.
df_clean = df_clean.fillna(0)

# (3) 변동이 아예 없는(표준편차가 0인) 종목이 있으면 상관계수가 nan이 됩니다. 이를 제거합니다.
df_clean = df_clean.loc[:, df_clean.std() > 0]

# (4) 이제 상관행렬을 계산하면 nan 없이 숫자가 나옵니다.
corr = df_clean.corr()

In [11]:
# 3. 고유값 분해 (RMT 적용 준비)
# 행렬에서 신호와 노이즈를 분리하기 위해 고유값을 구합니다.
eVal, eVec = LA.eigh(corr)

In [12]:
# 4. RMT(무작위 행렬 이론) 임계치 계산
# Marchenko-Pastur 분포를 사용하여 노이즈 범위를 정합니다.
T, N = df_clean.shape
q = T / N
sigma_sq = 1 - np.max(eVal) / N 
eMax = sigma_sq * (1 + (1/q)**0.5)**2

In [13]:
# 5. 클리핑(Clipping) 처리
# 임계치(eMax)보다 작은 고유값(노이즈)을 평균값으로 대체합니다.
eVal_clipped = eVal.copy()
low_values_indices = eVal < eMax
eVal_clipped[low_values_indices] = np.mean(eVal[low_values_indices])

In [14]:
# 6. 필터링된 상관행렬 재구성
# 노이즈가 제거된 깨끗한 상관행렬을 완성합니다.
corr_filtered = np.dot(eVec, np.dot(np.diag(eVal_clipped), eVec.T))
d = np.diag(corr_filtered)
corr_filtered = corr_filtered / np.sqrt(np.outer(d, d))

In [15]:
print("필터링된 상관행렬 생성이 완료되었습니다.")
print(corr_filtered)

필터링된 상관행렬 생성이 완료되었습니다.
[[ 1.          0.16123194 -0.03374613 ...  0.00353812  0.05116864
  -0.01411953]
 [ 0.16123194  1.         -0.12545157 ...  0.02586524  0.02655406
   0.01747966]
 [-0.03374613 -0.12545157  1.         ...  0.0219585  -0.00789375
   0.01725122]
 ...
 [ 0.00353812  0.02586524  0.0219585  ...  1.         -0.04523981
   0.15462939]
 [ 0.05116864  0.02655406 -0.00789375 ... -0.04523981  1.
   0.00112277]
 [-0.01411953  0.01747966  0.01725122 ...  0.15462939  0.00112277
   1.        ]]
